# Work 6.1: Pegasus-large fine-tuned on HUPD Descriptions

Fine tuning on HUPD descriptions, but with a smaller dataset because we had trouble with the last one crashing in colab.

(spoiler alert this one crashed too, but we did what we could with the model checkpoints)

Fine tuning code reference: HuggingFace [Transformers Notebooks](https://github.com/huggingface/transformers/tree/main/notebooks)

## Setup

In [ ]:
#install libraries
!pip install -q datasets
!pip install -q transformers
!pip install --quiet --upgrade accelerate
!pip install -q sentencepiece
!pip install -q evaluate
!pip install -q rouge_score
!pip install nltk

In [ ]:
# Install Packages

#standard data science libraries
import pandas as pd
import numpy as np
import random

#visualization
import matplotlib.pyplot as plt
from pprint import pprint
from IPython.display import display, HTML

#datasets
import datasets
from datasets import load_from_disk
from datasets import load_dataset, load_metric
from datasets import DatasetDict


#transformers
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from transformers import TrainingArguments
from transformers import AutoTokenizer

# PyTorch
import torch

#rouge
import evaluate
from evaluate import load

#nltk
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# This cell will authenticate you and mount your Drive in the Colab.
##### ensure you mount to the folder that you want.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
data_save_dir="/content/drive/MyDrive/W266/HUPD/"
model_save_dir="/content/drive/MyDrive/W266/ProjectModels/Pegasus_large_ft_descriptions_small/"

Mounted at /content/drive


##Load Data

In [ ]:
##########################for the test on small data
tokenized_dataset = load_from_disk(data_save_dir + 'HUPD_C07_tokenized_descriptions')

In [ ]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['patent_number', 'decision', 'title', 'abstract', 'claims', 'background', 'summary', 'description', 'cpc_label', 'ipc_label', 'filing_date', 'patent_issue_date', 'date_published', 'examiner_id', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 30915
    })
    validation: Dataset({
        features: ['patent_number', 'decision', 'title', 'abstract', 'claims', 'background', 'summary', 'description', 'cpc_label', 'ipc_label', 'filing_date', 'patent_issue_date', 'date_published', 'examiner_id', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 3655
    })
    test: Dataset({
        features: ['patent_number', 'decision', 'title', 'abstract', 'claims', 'background', 'summary', 'description', 'cpc_label', 'ipc_label', 'filing_date', 'patent_issue_date', 'date_published', 'examiner_id', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 3656
    })
})

In [ ]:
# val_test_split = dataset['validation'].train_test_split(test_size=0.5)

In [ ]:
# dataset['train'] = dataset['train']
# dataset['validation'] = val_test_split['train']
# dataset['test'] = val_test_split['test']

In [ ]:
#only run if you want to shrink the amount of data you are working with.
tokenized_subset = DatasetDict()
tokenized_subset['train'] = tokenized_dataset['train'].shuffle(seed=42).select(range(8000))
tokenized_subset['validation'] = tokenized_dataset['validation'].shuffle(seed=42).select(range(1000))
tokenized_subset['test'] = tokenized_dataset['test'].shuffle(seed=42).select(range(1000))

# Modeling

In [ ]:
model_checkpoint = "google/pegasus-large"

In [ ]:
metric = load('rouge')

### Tokenize Data

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/3.09k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

In [ ]:
max_input_length = 500 #any longer and we run out of memory
max_summary_length = 100

In [ ]:
# def preprocess_text(dataset, summarize_section = 'description'):
#   inputs = dataset[summarize_section]
#   model_inputs  = tokenizer(inputs, truncation = True, max_length = max_input_length) #, return_tensors = 'pt', , padding = 'max_length'
#   labels = dataset['abstract']
#   model_labels = tokenizer(text_target = labels, truncation = True, max_length = max_input_length) #, return_tensors = 'pt' , padding = 'max_length'
#   model_inputs['labels'] = model_labels['input_ids']
#   return model_inputs

In [ ]:
# tokenized_dataset = dataset.map(preprocess_text, batched=True)

Map:   0%|          | 0/30915 [00:00<?, ? examples/s]

Map:   0%|          | 0/3655 [00:00<?, ? examples/s]

Map:   0%|          | 0/3656 [00:00<?, ? examples/s]

In [ ]:
# tokenized_dataset.save_to_disk(data_save_dir + 'HUPD_C07_tokenized_descriptions')

Saving the dataset (0/12 shards):   0%|          | 0/30915 [00:00<?, ? examples/s]

Saving the dataset (0/2 shards):   0%|          | 0/3655 [00:00<?, ? examples/s]

Saving the dataset (0/2 shards):   0%|          | 0/3656 [00:00<?, ? examples/s]

### Set up Model and Fine Tune

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/260 [00:00<?, ?B/s]

In [ ]:
batch_size = 2
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-descriptions",
    evaluation_strategy = "epoch",
    learning_rate=10e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    # save_total_limit=6,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    # Note that other metrics may not have a `use_aggregator` parameter
    # and thus will return a list, computing a metric for each sentence.
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    # Extract a few results
    result = {key: value for key, value in result.items()} #(took out *100)

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_subset["train"],
    eval_dataset=tokenized_subset["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

You're using a PegasusTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,1.670500,1.531563,0.458300,0.323300,0.400000,0.419500,109.836000
2,1.852700,1.722241,0.455100,0.314100,0.398300,0.416000,76.265000


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,1.670500,1.531563,0.458300,0.323300,0.400000,0.419500,109.836000
2,1.852700,1.722241,0.455100,0.314100,0.398300,0.416000,76.265000


It crashed again :D
Last model checkpoint: Model checkpoint path: /content/pegasus-large-finetuned-descriptions/checkpoint-9500

In [ ]:
trainer.evaluate()

In [ ]:
test_predictions = trainer.predict(
                test_dataset = tokenized_dataset['test'],
                num_beams = 12,
                max_length = max_summary_length
                )

In [ ]:
test_predictions.metrics

In [ ]:
summarized_texts = []
for summary in test_predictions.predictions:
    summarized_text = tokenizer.batch_decode(summary, skip_special_tokens=False)
    summarized_texts.append(summarized_text)

In [ ]:
num_examples = 10
random_indices = np.random.choice(len(tokenized_dataset['test']), num_examples, replace=False)
print(random_indices)

In [ ]:
# Randomly sample example paies
sampled_summarized_texts = [summarized_texts[i] for i in random_indices]
sampled_abstracts = [tokenized_dataset['test']['abstract'][i] for i in random_indices]

# Zip the lists together
comparisons = list(zip(sampled_summarized_texts, sampled_abstracts))

# Create a pandas DataFrame from the zipped lists
comparison_df = pd.DataFrame(comparisons, columns=['predicted_summary', 'abstract'])

# Display df
display(HTML(comparison_df.to_html()))

In [ ]:
# del model
# del trainer
# torch.cuda.empty_cache()

## pulling together a test of the model from the model checkpoint:



In [ ]:
tokenized_subset['test'] = tokenized_dataset['test'].shuffle(seed=42).select(range(500))

In [ ]:
tokenized_subset['test']

Dataset({
    features: ['patent_number', 'decision', 'title', 'abstract', 'claims', 'background', 'summary', 'description', 'cpc_label', 'ipc_label', 'filing_date', 'patent_issue_date', 'date_published', 'examiner_id', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 500
})

In [ ]:
model = PegasusForConditionalGeneration.from_pretrained("/content/pegasus-large-finetuned-descriptions/checkpoint-9500")
tokenizer = PegasusTokenizer.from_pretrained("/content/pegasus-large-finetuned-descriptions/checkpoint-9500")

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
num_examples = 10
random_indices = np.random.choice(len(tokenized_subset['test']), num_examples, replace=False)
print(random_indices)

[264  57 153 278 483 409  22 213 169 476]


In [ ]:
#specify compute device (GPU if available)
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
#####er... we didnt run this....
#set format to Torch
tokenized_subset['test'].set_format(type='torch', columns=['input_ids', 'attention_mask'])

In [ ]:
batch_size = 10 #note that the standard batch size for the DataLoader is 200
test_dataloader = DataLoader(tokenized_subset['test'], batch_size = batch_size)

In [ ]:
summarized_texts = []
for batch in iter(test_dataloader):
    input_ids = batch['input_ids']#.to(device)
    attention_mask = batch['attention_mask'] #.to(device)
    summary = model.generate(input_ids = input_ids,
                             attention_mask = attention_mask,
                             max_length = 100,
                             num_beams = 12
                             )
    summarized_text = tokenizer.batch_decode(summary, skip_special_tokens=False)
    summarized_texts.extend(summarized_text)

In [ ]:
#Generate Rouge Scores
rouge = evaluate.load('rouge')
predictions = summarized_texts
references = tokenized_subset['test']['abstract']
claims_results = rouge.compute(predictions=predictions,
                        references=references)
print(claims_results)

{'rouge1': 0.2992458801143298, 'rouge2': 0.19531534423446176, 'rougeL': 0.2607563906021338, 'rougeLsum': 0.261211642093789}


In [ ]:
# Randomly sample example paies
sampled_summarized_texts = [summarized_texts[i] for i in random_indices]
sampled_abstracts = [tokenized_subset['test']['abstract'][i] for i in random_indices]

# Zip the lists together
comparisons = list(zip(sampled_summarized_texts, sampled_abstracts))

# Create a pandas DataFrame from the zipped lists
comparison_df = pd.DataFrame(comparisons, columns=['predicted_summary', 'abstract'])

# Display df
display(HTML(comparison_df.to_html()))

,predicted_summary,abstract
0,"<pad>The present invention relates to new hydroxyester derivatives, to a process for their preparation and to pharmaceutical compositions containing them.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>","Compounds of formula (I): wherein R1, R2, R3, R4, R5, R6, R7, R13, Ra, Rb, A and n are as defined in the description. Medicinal products containing the same which are useful in treating conditions requiring a pro-apoptotic agent."
1,"<pad>In some aspects, the disclosure provides novel AAVs and methods of use thereof as well as related kits.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>","The disclosure in some aspects relates to recombinant adeno-associated viruses having distinct tissue targeting capabilities. In some aspects, the disclosure relates to gene transfer methods using the recombinant adeno-associated viruses. In some aspects, the disclosure relates to isolated AAV capsid proteins and isolated nucleic acids encoding the same."
2,"<pad>Provided herein are pyridinethiones and pharmaceutical compositions thereof. Also provided herein are methods of their use for treating, preventing, or ameliorating one or more symptoms of a proliferative, inflammatory, neurodegenerative, or immune-mediated disease.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>","Provided herein are pyridinethiones, for example, a compound of Formula I, and pharmaceutical compositions thereof. Also provided herein are methods of their use for treating, preventing, or ameliorating one or more symptoms of a proliferative, inflammatory, neurodegenerative, or immune-mediated disease (e.g., multiple sclerosis)."
3,"<pad>Described herein are compounds of Formula (I), (Ia), (Ib), (Ic), (Id), (Ie), (Ig), (Ih), (Ii), (II), (IIa), (IIb), (IId), (IIe), (IIg), (IIh), (IIi), (IIa), (IIb), (IIc), (IId), (IIe), (IIg), (II</s>","Described herein are fused bicyclic compounds, compositions, and methods for their use for the treatment of disease."
4,"<pad>The invention relates to anti-cancer antibodies for targeting tumor cells expressing the glycolipid GD2. More specifically, the invention relates to anti-GD2 antibodies for targeting tumor cells expressing the glycolipid GD2.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>","The invention provides modified antibodies directed against GD2 that have diminished complement fixation relative to antibody-dependent, cell-mediated cytotoxicity, which is maintained. The modified antibodies of the invention may be used in the treatment of tumors such as neuroblastoma, glioblastoma, melanoma, small-cell lung carcinoma, B-cell lymphoma, renal carcinoma, retinoblastoma, and other cancers of neuroectodermal origin."
5,"<pad>The present invention relates to a compound of Formula (I) or a pharmaceutically acceptable salt thereof, wherein R1, R2, R3, and R4 are as defined herein. The present invention also relates to methods of using the compound of Formula (I) or a pharmaceutically acceptable salt thereof. The present invention also relates to methods of using the compound of Formula (I) or a pharmaceutically acceptable salt thereof, and methods of using the compound of Formula (I) or a pharmaceutically</s>","The present invention provides small